In [1]:
%cd drive/MyDrive/colab/distillog

/content/drive/MyDrive/colab/distillog


In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import string

def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message
    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [ ]:
%cd distillog/

/content/drive/MyDrive/colab/distillog


In [4]:
df = pd.read_csv("./datasets/Spirit5M/Spirit5M.log_templates.csv", header = None)

lines = df.values[:,1]
print(len(lines))

2881


In [5]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=0fad5cbbf20c0dfb15677bdeaeaba6ce9714db0c1bf3283f16b64952885a0434
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [6]:
import fasttext.util

In [ ]:
%cd ../

/content/drive/MyDrive/colab


In [7]:
ft = fasttext.load_model('../cc.en.300.bin')

In [8]:
s = lines[54]
print(s)

s = clean(s).lower()
print(s)

words = s.split(" ")
words = [word for word in words]
s = " ".join(words)

print(s)

79d3ba1c




In [9]:
s = '<*> <*> <*> <*> <*> stat=Deferred: Connection refused by [<*>]'
s = clean(s).lower()
print(s)

words = s.split(" ")
words = [word for word in words]
s = " ".join(words)

print(s)

stat deferred connection refused by
stat deferred connection refused by


In [10]:
def encoder(s):
    words = s.split(" ")
    words = [word for word in words]
    s = " ".join(words)
    vector = ft.get_sentence_vector(s)
    return vector

In [11]:
clean_template_vector = []
vec = []



for i, line in enumerate(lines):
    content = clean(line).lower()
    vec.append(encoder(content))


data_df = pd.DataFrame(vec)


In [12]:
print(len(vec))

2881


In [13]:
from sklearn.decomposition import PCA
import numpy as np

estimator30 = PCA(n_components=30)
pca_30 = estimator30.fit_transform(vec)

# Step1-3 PPA: De-averaged
vec30 = []
result = pca_30 - np.mean(pca_30)
pca = PCA(n_components=30)
pca_30 = pca.fit_transform(result)
U = pca.components_
for i, x in enumerate(result):
    for u in U[0:7]:
        x = x - np.dot(u.transpose(), x) * u
    vec30.append(list(x))
vec30 = np.array(vec30)

In [14]:
print(len(vec30))

2881


In [15]:
data_df.to_csv("./datasets/Spirit5M/vector.csv", header = None, index = None)

data_pca_df = pd.DataFrame(vec30)
data_pca_df.to_csv("./datasets/Spirit5M/pca_vector.csv", header = None, )